<h1>Покупка квартиры мечты</h1>
<img src="media/dream_apartment.jpg/">

Вводные:
1. Нужно купить квартирку, в которой мы будем жить и которая выгодна с точки зрения инвестиций.
2. Будет ипотека. Ограничения: первоначальный взнос 1.8 млн рублей, 5 лет, платеж до 200 тыр.


<span style="color:#ffffff; background: #0703fc">Цель исследования: сравнить рост цены недвижки и переплату по ипотеке</span>

План анализа:
1. Изучить расчет платежей по ипотеке. Аннуитетные платежи. Досрочные платежи.
2. Выбрать и распарсить данные по недвиге - собрать датасет.
4. Найти старый парсинг циана, посмотреть, как растут в цене новостройки по районам и застройщикам.
5. Посчитать что выгоднее.

<h2>Функции для расчета ипотеки</h2>

In [53]:
import pandas as pd

# считаем платеж и переплату при аннуитетном платеже
# нужно передать датафрейм с полями
# стоимость хаты (s), первоначальный взнос (fst), срок ипотеки в месяцах (n), годовая ставка (p)
def annuit(str):
    p = str.p / 12
    debt = str.s - str.fst
    payment = debt * (p + p / (pow((1 + p), str.n) - 1))
    overpay = payment * str.n - debt
    return payment, overpay

# возвращает график платежей
# нужно передать датафрейм с полями
# стоимость хаты (s), первоначальный взнос (fst), срок ипотеки в месяцах (n), годовая ставка (p)
def payment_schedule(str):
    payments_list = []
    rest = str.s - str.fst
    n = int(str.n)
    i = 1
    while (i <= n) & (rest > 0) :
        perc_mon = rest * str.p/12
        real_payment = str.payment
        if real_payment > rest:
            real_payment = rest + perc_mon
        debt_mon = real_payment - perc_mon
        rest = rest - debt_mon
        payments_list.append([i, real_payment, rest, perc_mon, debt_mon])
        # print(i, real_payment, rest, perc_mon, debt_mon)
        i+=1
    df = pd.DataFrame(payments_list)
    df = df.rename(columns={0:'i', 1:'payment', 2:'rest', 3:'perc_mon', 4:'debt_mon'})
    return df

# возвращает график платежей с досрочным погашением
# нужно передать датафрейм с полями
# стоимость хаты (s), первоначальный взнос (fst), срок ипотеки в месяцах (n), годовая ставка (p)
# и список досрочных погашений с полями [[0, 1, 2, 3]]
# сколько платить (0), периодичность (1) 0 - разово, 1 - раз в месяц, 2 - раз в 2 месяца и тд, начало (2), конец (3)
# можно несколько досрочных погашений
def payment_schedule_early(str, eps):
    payments_list = []
    rest = str.s - str.fst
    n = int(str.n)
    i = 1
    while (i <= n) & (rest > 0) :
        perc_mon = rest * str.p/12
        real_payment = str.payment
        for ep in eps:
            if i in range(ep[2], ep[3]):
                if (ep[1] == 0)|((i - ep[2]) % ep[1] == 0):
                    real_payment = ep[0]
        if real_payment > rest:
            real_payment = rest + perc_mon
        debt_mon = real_payment - perc_mon
        rest = rest - debt_mon
        payments_list.append([i, real_payment, rest, perc_mon, debt_mon])
        i+=1
    df = pd.DataFrame(payments_list)
    df = df.rename(columns={0:'i', 1:'payment', 2:'rest', 3:'perc_mon', 4:'debt_mon'})
    return df

Посчитаем переплату по желаемой хате с разными параметрами

In [43]:
# стоимость хаты, первоначальный взнос, срок ипотеки в месяцах, годовая ставка
params = [(12189868, 12189868*0.15, 60, 0.045),
          (12189868, 12189868*0.15, 36, 0.045),
          (12189868, 12189868*0.15, 240, 0.045),
          (12189868, 12189868*0.15, 360, 0.045)]

In [45]:
import pandas as pd

diff = pd.DataFrame(params)
diff = diff.rename(columns={0:'s', 1:'fst', 2:'n', 3:'p'})
diff['debt'] = diff['s'] - diff['fst']
diff[['payment', 'overpay']] =  diff.apply(lambda row: annuit(row), axis='columns', result_type='expand')
diff.style.format({'fst':'{:.2f}','p': '{:.3f}', 'debt':'{0:_}', 'payment':'{0:_.2f}', 'overpay':'{0:_.2f}'})

,s,fst,n,p,debt,payment,overpay
0,12189868,1828480.20,60,0.045,10_361_387.8,193_167.55,1_228_665.33
1,12189868,1828480.20,36,0.045,10_361_387.8,308_219.42,734_511.33
2,12189868,1828480.20,240,0.045,10_361_387.8,65_551.26,5_370_913.47
3,12189868,1828480.20,360,0.045,10_361_387.8,52_499.63,8_538_478.96


Кредит дадут на миллиард лет, но гасить лучше досрочно. Посчитаем, лучше сразу ебашить по 200 тыр или можно начать через 3 месяца.

In [85]:
# сколько платить, периодичность (0 - разово, 1 - раз в месяц, 2 - раз в 2 месяца и тд), начало, конец
early_payments =[[200000, 1, 4, 10000]]
payments = payment_schedule_early(diff.loc[2],early_payments)
overpay1 = payments.perc_mon.sum()
print('Overpay = ', overpay1)
print('n = ',payments.shape[0])

# сколько платить, периодичность (0 - разово, 1 - раз в месяц, 2 - раз в 2 месяца и тд), начало, конец
early_payments =[[200000, 1, 1, 10000]]
payments = payment_schedule_early(diff.loc[2],early_payments)
overpay2 = payments.perc_mon.sum()
print('Overpay = ', overpay2)
print('n = ',payments.shape[0])
print(overpay1-overpay2)

Overpay =  1277800.250170369
n =  61
Overpay =  1181026.4180786763
n =  58
96773.8320916926


ПЕРЕПЛАТА БУДЕТ СТОИТЬ 100 ТЫЩ, ГАСИМ СРАЗУ

График платежей тогда такой

In [87]:
payments.style.format({'rest':'{0:_.2f}', 'perc_mon':'{:.2f}', 'debt_mon':'{:.2f}'})

,i,payment,rest,perc_mon,debt_mon
0,1,200000.000000,10_200_243.00,38855.20,161144.80
1,2,200000.000000,10_038_493.92,38250.91,161749.09
2,3,200000.000000,9_876_138.27,37644.35,162355.65
3,4,200000.000000,9_713_173.79,37035.52,162964.48
4,5,200000.000000,9_549_598.19,36424.40,163575.60
5,6,200000.000000,9_385_409.18,35810.99,164189.01
6,7,200000.000000,9_220_604.47,35195.28,164804.72
7,8,200000.000000,9_055_181.73,34577.27,165422.73
8,9,200000.000000,8_889_138.66,33956.93,166043.07
9,10,200000.000000,8_722_472.93,33334.27,166665.73


<h2>Парсим сайты с недвигой</h2>

In [1]:
import requests
from bs4 import BeautifulSoup # for web scraping
import pandas as pd
import numpy as np
import re
from ipywidgets import IntProgress #for progress bar
from IPython.display import display

In [70]:
req = requests.get('https://m2.ru/sankt-peterburg/nedvizhimost/kupit-kvartiru/')
soup = BeautifulSoup(req.text, "lxml")

In [71]:
all_hrefs = [a.get('href') for a in soup.find_all('a')] #list of all links
flat_hrefs=[a for a in all_hrefs if 'https://m2.ru/sankt-peterburg/nedvizhimost/prodazha' in str(a)] #filtering the list
flat_hrefs=list(set(flat_hrefs)) #removing duplicates in the list
flat_hrefs[0:5]

['https://m2.ru/sankt-peterburg/nedvizhimost/prodazha-1-komnatnoi-kvartiri-53-3-m-1-6-etazh-672d9b9c-8827-42ca-b7be-736115ce772e/',
 'https://m2.ru/sankt-peterburg/nedvizhimost/prodazha-1-komnatnoi-kvartiri-37-1-m-3-9-etazh-c886fe91-b6cb-4533-b7ba-688773fd11fc/',
 'https://m2.ru/sankt-peterburg/nedvizhimost/prodazha-2-komnatnoi-kvartiri-67-4-m-6-12-etazh-6ac17150-d91b-4273-af15-f659059d7901/',
 'https://m2.ru/sankt-peterburg/nedvizhimost/prodazha-2-komnatnoi-kvartiri-63-1-m-8-8-etazh-d9d2f232-05ad-48d2-819e-1bfdd7e4e12a/',
 'https://m2.ru/sankt-peterburg/nedvizhimost/prodazha-1-komnatnoi-kvartiri-33-9-m-8-14-etazh-bf400424-6e48-43de-be62-ea1e6ba5b4e7/']

In [72]:
def add_flats(hrefs):
    flats = pd.DataFrame()
    progressbar=IntProgress(min=0, max=len(hrefs), value=0) #progress bar
    display(progressbar)
    for i,href in enumerate(hrefs):
        # getting html code of the flat page
        try:
            req1 = requests.get(href)
        except requests.exceptions.RequestException as e:
            raise SystemExit(e)
        soup = BeautifulSoup(req1.text, "lxml")
        flat = {'Ссылка': href,
               'Название': soup.h1.text.replace('\xa0',' '),
               'Адрес': ', '.join(a.string for a in soup.find_all(attrs={"class": "ClClickableAddress__link"})),
               'Цена': soup.find(attrs={"itemprop": "price"}).text.replace('\xa0',' ')}
        attr = soup.find_all(attrs={"class":"colors-named-module__secondary___2WPH_ fonts-module__secondary-alone___3wVR-"})
        vals = soup.find_all(attrs={"class":"fonts-module__primary___2PNSt DescriptionCell__value"})
        for a,v in zip(attr,vals):
            flat[a.text] = v.text.replace('\xa0',' ')
            progressbar.value = i+1
        flats = pd.concat([flats, pd.DataFrame([flat])], ignore_index=True)
    return flats

In [77]:
flats_all = pd.DataFrame()

In [130]:
n_page=373
while n_page<=500:
    print('page number:',str(n_page))
    try:
        req = requests.get('https://m2.ru/sankt-peterburg/nedvizhimost/kupit-kvartiru/?pageNumber='+str(n_page))
    except requests.exceptions.RequestException as e:
        raise SystemExit(e)

    soup = BeautifulSoup(req.text, "lxml")
    # if we don't have header we stop parsing
    if soup.h1 is None:
        print('Parsing is stopped')
        break
    #getting all links from the page
    all_hrefs = [a.get('href') for a in soup.find_all('a')]
    flat_hrefs = [a for a in all_hrefs if 'https://m2.ru/sankt-peterburg/nedvizhimost/prodazha' in str(a)]
    #getting all parameters from an apartment page.
    flat_hrefs = list(set(flat_hrefs))
    flats = add_flats(flat_hrefs)
    flats_all = pd.concat([flats_all, flats], ignore_index=True)
    print('number of rows',flats_all.shape[0])
    n_page+=1

page number: 373


IntProgress(value=0, max=20)

number of rows 7460
page number: 374


IntProgress(value=0, max=20)

number of rows 7480
page number: 375


...

number of rows 9960
page number: 499


IntProgress(value=0, max=20)

number of rows 9980
page number: 500


IntProgress(value=0, max=20)

number of rows 10000


In [131]:
flats_all

,Ссылка,Название,Адрес,Цена,Комнат,Отделка,Площадь квартиры,Жилая площадь,Площадь кухни,Этаж,...,Кухонная мебель,Телевизор,Стиральная машина,Год постройки,Консьерж,Материал стен,Отопление,Вид из окон,Серия дома,Временная эпоха
0,https://m2.ru/sankt-peterburg/nedvizhimost/pro...,"Продажа 1-комнатной квартиры 34 м², 9/12 этаж","Санкт-Петербург, Выборгский район, Большой Сам...",10 500 000,1,чистовая,34 м²,"16,2 м²","10,1 м²",9 из 12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://m2.ru/sankt-peterburg/nedvizhimost/pro...,"Продажа 1-комнатной квартиры 53,3 м², 1/6 этаж","Санкт-Петербург, Петроградский район, Петровск...",32 700 000,1,NaN,"53,3 м²",NaN,NaN,1 из 6,...,есть,есть,есть,2018,есть,NaN,NaN,NaN,NaN,NaN
2,https://m2.ru/sankt-peterburg/nedvizhimost/pro...,"Продажа 1-комнатной квартиры 37,1 м², 3/9 этаж","Санкт-Петербург, Невский район, проспект Обухо...",11 750 289,1,чистовая,"37,1 м²","17,3 м²","10,1 м²",3 из 9,...,NaN,NaN,NaN,NaN,NaN,кирпич-монолит,NaN,NaN,NaN,NaN
3,https://m2.ru/sankt-peterburg/nedvizhimost/pro...,"Продажа 2-комнатной квартиры 67,4 м², 6/12 этаж","Санкт-Петербург, Красносельский район, Ленинск...",12 500 000,2,NaN,"67,4 м²","34,4 м²","15,4 м²",6 из 12,...,есть,NaN,есть,NaN,есть,монолит,центральное,NaN,NaN,NaN
4,https://m2.ru/sankt-peterburg/nedvizhimost/pro...,"Продажа 2-комнатной квартиры 63,1 м², 8/8 этаж","Санкт-Петербург, Московский район, Черниговска...",17 490 000,2,NaN,"63,1 м²","33,2 м²","10,2 м²",8 из 8,...,NaN,NaN,NaN,2023,NaN,кирпич-монолит,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,https://m2.ru/sankt-peterburg/nedvizhimost/pro...,"Продажа квартиры-студии 30,2 м², 5/23 этаж","Санкт-Петербург, Выборгский район, Парголово, ...",6 447 519,NaN,предчистовая,"30,2 м²","19,1 м²",NaN,5 из 23,...,NaN,NaN,NaN,2022,NaN,NaN,NaN,NaN,NaN,NaN
9996,https://m2.ru/sankt-peterburg/nedvizhimost/pro...,"Продажа квартиры-студии 33,1 м², 15/25 этаж","Санкт-Петербург, Приморский район, Богатырский...",8 400 000,NaN,NaN,"33,1 м²","21,4 м²",NaN,15 из 25,...,NaN,NaN,NaN,2022,NaN,кирпич-монолит,NaN,NaN,NaN,NaN
9997,https://m2.ru/sankt-peterburg/nedvizhimost/pro...,"Продажа квартиры-студии 30,4 м², 4/6 этаж","Санкт-Петербург, Московский район, улица Коли ...",7 476 000,NaN,NaN,"30,4 м²","17,9 м²",NaN,4 из 6,...,NaN,NaN,NaN,NaN,NaN,кирпич,NaN,NaN,NaN,NaN
9998,https://m2.ru/sankt-peterburg/nedvizhimost/pro...,"Продажа 2-комнатной квартиры 45 м², 6/9 этаж","Санкт-Петербург, Выборгский район, Выборгское ...",6 600 000,2,NaN,45 м²,"27,5 м²","6,4 м²",6 из 9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [132]:
flats_all.to_csv('data/raw_data_from_metr2_2023.csv',sep=';',index=False)

<h2>Почистим данные</h2>

In [1]:
import pandas as pd
flats = pd.read_csv('data/raw_data_from_metr2_2023.csv',sep=';')

In [2]:
flats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 36 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Ссылка             10000 non-null  object 
 1   Название           10000 non-null  object 
 2   Адрес              10000 non-null  object 
 3   Цена               10000 non-null  object 
 4   Комнат             8187 non-null   float64
 5   Отделка            4002 non-null   object 
 6   Площадь квартиры   10000 non-null  object 
 7   Жилая площадь      9571 non-null   object 
 8   Площадь кухни      7588 non-null   object 
 9   Этаж               9999 non-null   object 
 10  Санузел            7270 non-null   object 
 11  Кол-во санузлов    7599 non-null   float64
 12  Балкон/лоджия      6892 non-null   object 
 13  Вид из окна        4234 non-null   object 
 14  Срок сдачи         4175 non-null   object 
 15  Высота потолков    7228 non-null   object 
 16  Лифт               6180

In [3]:
flats['Адрес'].head()

0    Санкт-Петербург, Выборгский район, Большой Сам...
1    Санкт-Петербург, Петроградский район, Петровск...
2    Санкт-Петербург, Невский район, проспект Обухо...
3    Санкт-Петербург, Красносельский район, Ленинск...
4    Санкт-Петербург, Московский район, Черниговска...
Name: Адрес, dtype: object

In [99]:
# попробуем split
flats['Адрес'].str.split(',', expand=True)

,0,1,2,3,4,5
0,Санкт-Петербург,Выборгский район,Большой Сампсониевский проспект,7,None,None
1,Санкт-Петербург,Петроградский район,Петровский проспект,2с2,None,None
2,Санкт-Петербург,Невский район,проспект Обуховской Обороны,None,None,None
3,Санкт-Петербург,Красносельский район,Ленинский проспект,72к1,None,None
4,Санкт-Петербург,Московский район,Черниговская улица,11к1,None,None
...,...,...,...,...,...,...
9995,Санкт-Петербург,Выборгский район,Парголово,Ольгинская дорога,3/20,None
9996,Санкт-Петербург,Приморский район,Богатырский проспект,2А,None,None
9997,Санкт-Петербург,Московский район,улица Коли Томчака,10,None,None
9998,Санкт-Петербург,Выборгский район,Выборгское шоссе,2,None,None


Как раскидать адрес:
- первая колонка - всегда город
- вторая колонка - всегда район
- улицы и номера дома может не быть
- улица может быть в последней заполненной колонке или в колонке до номера дома
- номер дома всегда последний и содержит цифры
- все остальное не нужно

In [138]:
import re
temp_addr = flats['Адрес'].str.split(',')
flats['Город'] = [a[0] for a in temp_addr]
flats['Район'] = [a[1] for a in temp_addr]
temp_addr = [a[2:] for a in temp_addr]
flats['Улица'] = ['' if len(a)==0 else a[0] if len(a)==1 else a[-2] for a in temp_addr]
flats['Номер дома'] = [a[-1] if len(a)>0 else '' for a in temp_addr]
flats['Номер дома'] = flats['Номер дома'].apply(lambda a:a if bool(re.search(r'\d',a)) else '')

In [139]:
flats[['Адрес', 'Город', 'Район', 'Улица', 'Номер дома']]

,Адрес,Город,Район,Улица,Номер дома
0,"Санкт-Петербург, Выборгский район, Большой Сам...",Санкт-Петербург,Выборгский район,Большой Сампсониевский проспект,7
1,"Санкт-Петербург, Петроградский район, Петровск...",Санкт-Петербург,Петроградский район,Петровский проспект,2с2
2,"Санкт-Петербург, Невский район, проспект Обухо...",Санкт-Петербург,Невский район,проспект Обуховской Обороны,
3,"Санкт-Петербург, Красносельский район, Ленинск...",Санкт-Петербург,Красносельский район,Ленинский проспект,72к1
4,"Санкт-Петербург, Московский район, Черниговска...",Санкт-Петербург,Московский район,Черниговская улица,11к1
...,...,...,...,...,...
9995,"Санкт-Петербург, Выборгский район, Парголово, ...",Санкт-Петербург,Выборгский район,Ольгинская дорога,3/20
9996,"Санкт-Петербург, Приморский район, Богатырский...",Санкт-Петербург,Приморский район,Богатырский проспект,2А
9997,"Санкт-Петербург, Московский район, улица Коли ...",Санкт-Петербург,Московский район,улица Коли Томчака,10
9998,"Санкт-Петербург, Выборгский район, Выборгское ...",Санкт-Петербург,Выборгский район,Выборгское шоссе,2
